In [1]:
import tensorflow as tf
import numpy as np
import tqdm

Num_of_Unique_Song=130009
Num_of_Unique_Tag=16141
data_dir = 'data/'

In [2]:
idx_Song = np.load(data_dir + 'idx_Song.npy')
idx_Tag = np.load(data_dir + 'idx_Tag.npy')
print(idx_Song.shape)
print(idx_Tag.shape)

(457051, 6)
(457051, 6, 3)


In [3]:
Song_X = idx_Song[:100,0:5]
print(Song_X.shape)
Song_Y = idx_Song[:100,5:6]
print(Song_Y.shape)
Tag_X = idx_Tag[:100,0:5,:]
print(Tag_X.shape)



(100, 5)
(100, 1)
(100, 5, 3)


In [4]:
Fake_Song = []
#sample one hot data
for _ in range(100):
    Fake_Song.append([])
    for _ in range(5):
        random = np.random.randint(Num_of_Unique_Song)
        #print(random)
        Fake_Song[len(Fake_Song)-1].append(random)
Fake_Song = np.array(Fake_Song)
print(Fake_Song.shape)

(100, 5)


In [5]:
Fake_Tag = []
#sample one hot data
for _ in range(100):
    Fake_Tag.append([])
    for _ in range(5):
        Fake_Tag[len(Fake_Tag)-1].append([])
        for _ in range(3):
            random = np.random.randint(Num_of_Unique_Tag)
            #print(random)
            Fake_Tag[len(Fake_Tag)-1][len(Fake_Tag[len(Fake_Tag)-1])-1].append(random)
Fake_Tag = np.array(Fake_Tag)
print(Fake_Tag.shape)

(100, 5, 3)


In [6]:
Fake_Y = []
#sample one hot data
for _ in range(100):
    Fake_Y.append([])
    for _ in range(1):
        random = np.random.randint(Num_of_Unique_Song)
        #print(random)
        Fake_Y[len(Fake_Y)-1].append(random)
Fake_Y = np.array(Fake_Y)
print(Fake_Y.shape)

(100, 1)


In [4]:
class Song_Model:
    def __init__(self):
        # Data Size
        self.song_size = 130009
        self.num_song = 5
        # Data
        self.input_x = tf.placeholder(tf.int32, shape=[None, self.num_song], name='input_x')
        # embedding output length
        self.song_embedding_len = 50
        # GRU setting
        self.gru_neuron = 50
        self.gru_size = 1
        self.sequence_length = [5]
        # Attention setting
        self.attention_size = 1
        # Drop out
        self.keep_prob = tf.placeholder(tf.float32, name='song_keep_prob')
        
        # Song one hot layer
        with tf.variable_scope('SONG_ONE_HOT'):
            onehot_input = tf.one_hot(self.input_x, self.song_size, name='song_one_hot')
            onehot_input = tf.reshape(onehot_input, [self.num_song, self.song_size])
        print('onehot_input')
        print(onehot_input.shape)
            
        # Song embedding layer
        with tf.variable_scope('SONG_EMBEDDING'):
            self.embedding = tf.Variable(tf.truncated_normal([self.song_size, self.song_embedding_len], stddev=0.1), 
                                         name='song_embedding_weight')
            embedding_song = []
            for s in range(self.num_song):
                embedding_outputs = tf.reshape(tf.matmul(onehot_input[s:s+1, :], self.embedding), 
                                           [1, 1, self.song_embedding_len])
                embedding_song.append(embedding_outputs)
            embedding_outputs = tf.concat(embedding_song,axis=1)
        print('embedding_outputs')
        print(embedding_outputs.shape)
        
        
        # Bi-GRU
        # Define Forward RNN Cell
        with tf.variable_scope('SONG_GRU_FORWARD') as scope:
            fw_rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(self.gru_size) 
                                                       for _ in range(self.gru_neuron)])
            fw_rnn_cell = tf.contrib.rnn.DropoutWrapper(fw_rnn_cell, output_keep_prob=self.keep_prob)
        
        # Define Backward RNN Cell
        with tf.variable_scope('SONG_GRU_BACKWARD') as scope:
            bw_rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(self.gru_size) 
                                                       for _ in range(self.gru_neuron)])
            bw_rnn_cell = tf.contrib.rnn.DropoutWrapper(bw_rnn_cell, output_keep_prob=self.keep_prob)
        
        with tf.variable_scope('SONG_Bi_GRU'):
            # embedding_inputs shape: (batch_size, sequence_length, embedding_size)
            # rnn_output, _ = tf.nn.dynamic_rnn(fw_rnn_cell, inputs=embedding_inputs, 
            #                                   sequence_length=self.seq_len, dtype=tf.float32)
            rnn_output, _ = tf.nn.bidirectional_dynamic_rnn(fw_rnn_cell, bw_rnn_cell, 
                                                            inputs=embedding_outputs, 
                                                            sequence_length=self.sequence_length, 
                                                            dtype=tf.float32)
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs
        if isinstance(rnn_output, tuple):
            bigru_output = tf.concat(rnn_output, 2)
        print('bigru_output')
        print(bigru_output.shape)
        
        # Attention Layer
        with tf.name_scope('SONG_ATTENTION'):
            input_shape = bigru_output.shape # (batch_size, sequence_length, hidden_size)
            sequence_size = input_shape[1].value  # the length of sequences processed in the RNN layer
            hidden_size = input_shape[2].value  # hidden size of the RNN layer
            #print(hidden_size)

            attention_w = tf.Variable(tf.truncated_normal([hidden_size, self.attention_size], stddev=0.1), 
                                      name='song_attention_weight')
            #attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
            #attention_u = tf.Variable(tf.truncated_normal([attention_size], stddev=0.1), name='attention_u')
            z_list = []
            for t in range(sequence_size):
                u_t = tf.tanh(tf.matmul(bigru_output[:, t, :], attention_w))
                #z_t = tf.matmul(u_t, tf.reshape(attention_u, [-1, 1]))
                z_list.append(u_t)
            # Transform to batch_size * sequence_size
            self.attention_output = tf.concat(z_list, axis=1)
            #self.alpha = tf.nn.softmax(attention_z)
            # Transform to batch_size * sequence_size * 1 , same rank as bigru_output
            #self.attention_output = tf.reduce_sum(bigru_output * tf.reshape(self.attention_z, 
            #                                                                [-1, sequence_size, 1]), 1)
        print('attention_output')
        print(self.attention_output.shape)

In [5]:
class Tag_Model:
    def __init__(self):
        # Data Size
        self.tag_size = 16141
        self.num_song = 5
        self.num_tag = 3
        # Data
        self.input_x = tf.placeholder(tf.int32, shape=[None, self.num_song, self.num_tag], name='input_x')
        # Embedding setting
        self.num_tag_list = [3,2,1,1,2]
        self.tag_embedding_len = 25
        # GRU setting
        self.gru_neuron = 50
        self.gru_size = 5
        self.sequence_length = [5]
        # Attention setting
        self.attention_size = 1
        
        # Drop out
        self.keep_prob = tf.placeholder(tf.float32, name='tag_keep_prob')
        #learning rate
        self.lr = 0.05
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        #self.g_params = []
        
        # Tag one hot layer
        with tf.variable_scope('Tag_ONE_HOT'):
            onehot_input = tf.one_hot(self.input_x, self.tag_size, name='tag_one_hot')
            onehot_input = tf.reshape(onehot_input, [self.num_song, self.num_tag, self.tag_size])
        print('onehot_input')
        print(onehot_input.shape)
        
        #tag embedding layer
        with tf.variable_scope('TAG_EMBEDDING'):
            self.embedding = tf.Variable(tf.truncated_normal([self.tag_size, self.tag_embedding_len], stddev=0.1), 
                                         name='tag_embedding_weight')
            embedding_tag = []
            for s in range(self.num_song):
                for num in self.num_tag_list:
                    sum_embedding_tag = []
                    for t in range(num):
                        tag_vector = tf.matmul(tf.reshape(onehot_input[s:s+1,t:t+1,:],[1, self.tag_size]), 
                                                  self.embedding)
                        sum_embedding_tag.append(tag_vector)
                embedding_tag.append(tf.reduce_mean(sum_embedding_tag, 0))
            embedding_outputs = tf.concat(embedding_tag,axis=0)
            embedding_outputs = tf.reshape(embedding_outputs, [1, self.num_song, self.tag_embedding_len])
        print('embedding_outputs')
        print(embedding_outputs.shape)
        
        #Bi-GRU
        # Define Forward RNN Cell
        with tf.variable_scope('TAG_GRU_FORWARD') as scope:
            fw_rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(self.gru_size) 
                                                       for _ in range(self.gru_neuron)])
            fw_rnn_cell = tf.contrib.rnn.DropoutWrapper(fw_rnn_cell, output_keep_prob=self.keep_prob)
        
        # Define Backward RNN Cell
        with tf.variable_scope('TAG_GRU_BACKWARD') as scope:
            bw_rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(self.gru_size) 
                                                       for _ in range(self.gru_neuron)])
            bw_rnn_cell = tf.contrib.rnn.DropoutWrapper(bw_rnn_cell, output_keep_prob=self.keep_prob)
        
        with tf.variable_scope('TAG_Bi_GRU'):
            # embedding_inputs shape: (batch_size, sequence_length, embedding_size)
            # rnn_output, _ = tf.nn.dynamic_rnn(fw_rnn_cell, inputs=embedding_inputs, 
            #                                   sequence_length=self.seq_len, dtype=tf.float32)
            rnn_output, _ = tf.nn.bidirectional_dynamic_rnn(fw_rnn_cell, bw_rnn_cell, 
                                                            inputs=embedding_outputs, 
                                                            sequence_length=self.sequence_length, 
                                                            dtype=tf.float32)
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs
        if isinstance(rnn_output, tuple):
            bigru_output = tf.concat(rnn_output, 2)
        print('bigru_output')
        print(bigru_output.shape)
        
        # Attention Layer
        with tf.name_scope('TAG_ATTENTION'):
            input_shape = bigru_output.shape # (batch_size, sequence_length, hidden_size)
            sequence_size = input_shape[1].value  # the length of sequences processed in the RNN layer
            hidden_size = input_shape[2].value  # hidden size of the RNN layer
            #print(hidden_size)

            attention_w = tf.Variable(tf.truncated_normal([hidden_size, self.attention_size], stddev=0.1), 
                                      name='tag_attention_weight')
            #attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
            #attention_u = tf.Variable(tf.truncated_normal([attention_size], stddev=0.1), name='attention_u')
            z_list = []
            for t in range(sequence_size):
                u_t = tf.tanh(tf.matmul(bigru_output[:, t, :], attention_w))
                #z_t = tf.matmul(u_t, tf.reshape(attention_u, [-1, 1]))
                z_list.append(u_t)
            # Transform to batch_size * sequence_size
            self.attention_output = tf.concat(z_list, axis=1)
            #self.alpha = tf.nn.softmax(attention_z)
            # Transform to batch_size * sequence_size * 1 , same rank as bigru_output
            #self.attention_output = tf.reduce_sum(bigru_output * tf.reshape(self.attention_z, [-1, sequence_size, 1]), 1)
        print('attention_output')
        print(self.attention_output.shape)    

In [6]:
class ALL_Model:
    def __init__(self):
        self.Song_Part = Song_Model()
        self.Tag_Part = Tag_Model()
        self.FC_input = tf.concat(axis=1, values=[self.Song_Part.attention_output, 
                                                  self.Tag_Part.attention_output])
        #self.FC_input = self.Song_Part.attention_output
        print('FC_input')
        print( self.FC_input.shape)
        
        # fully connected setting
        self.fully_input_d = self.FC_input.get_shape().as_list()[1]
        self.fully_neuron = 50
        # output setting
        self.outpu_d = 130009
        
        
        
        # Fully connected layer
        with tf.name_scope('FULLY_CONNECTED_LAYER'):
            fully_weight = tf.Variable(tf.truncated_normal([self.fully_input_d, self.fully_neuron], 
                                                           stddev=0.1), name='fully_weight')
            fully_bais = tf.Variable(tf.constant(0.1, shape=[1, self.fully_neuron]), name='fully_bais')
        FC_output = tf.nn.relu(tf.matmul(self.FC_input, fully_weight) + fully_bais)
        print('FC_output')
        print(FC_output.shape)
        
        # Output layer
        with tf.name_scope('OUTPUT_LAYER'):
            fully_weight = tf.Variable(tf.truncated_normal([self.fully_neuron, self.outpu_d], 
                                                           stddev=0.1), name='Output_weight')
            fully_bais = tf.Variable(tf.constant(0.1, shape=[1, self.outpu_d]), name='Output_bais')
        self.Output = tf.nn.softmax(tf.matmul(FC_output, fully_weight) + fully_bais)
        print('Output')
        print(self.Output.shape)

In [9]:
Model = tf.Graph()
tf.reset_default_graph()
with Model.as_default():
    tf_config = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False)
    #tf_config.per_process_gpu_memory_fraction = 0.8
    tf_config.gpu_options.allow_growth = True
    with tf.Session(config=tf_config).as_default() as sess:
        model = ALL_Model()
        learning_rate = 0.5
        grad_clip = 5.0
        num_predict = 1
        Num_of_Unique_Song=130009
        Num_of_Unique_Tag=16141
        input_y = tf.placeholder(tf.int32, shape=[None, num_predict], name='input_y')
        # Label one hot layer
        with tf.variable_scope('LABEL_ONE_HOT'):
            onehot_label = tf.one_hot(input_y, Num_of_Unique_Song, name='label_one_hot')
            onehot_label = tf.reshape(onehot_label, [num_predict, Num_of_Unique_Song])
            print('onehot_label')
            print(onehot_label.shape)
        
        # Calculate cross-entropy loss
        with tf.name_scope('loss'):
            log_loss = tf.losses.softmax_cross_entropy(logits=model.Output, onehot_labels=onehot_label)
            loss = tf.reduce_mean(log_loss)
            
        # Create optimizer
        with tf.name_scope('optimization'):
            optimizer = tf.train.AdamOptimizer(learning_rate)
            
            train_op = optimizer.minimize(loss)
            
            #gradients, variables = zip(*optimizer.compute_gradients(loss))
            #gradients, _ = tf.clip_by_global_norm(gradients, grad_clip)
            #train_op = optimizer.apply_gradients(zip(gradients, variables))

        # Calculate accuracy
        with tf.name_scope('accuracy'):
            predictions = tf.argmax(model.Output, 1, name='predictions')
            correct_pred = tf.equal(predictions, tf.argmax(onehot_label, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
    
        
        for epoch in range(5):
            #random
            random_size = 1
            for i in tqdm.tqdm(range(Song_X.shape[0]//random_size)):
                if random_size == 1:
                    start = i
                    end = i + 1
                else:
                    idx = np.random.randint(Song_X.shape[0]) - 1
                    start = min(idx, Song_X.shape[0] - 1)
                    end = min(idx + 1, Song_X.shape[0])

                Song_input = Song_X[start:end,:]
                Tag_input = Tag_X[start:end,:,:]
                Song_label = Song_Y[start:end,:]
                
                feed_dict = {model.Song_Part.input_x: Song_input, 
                             #model.Tag_Part.input_x: Tag_input, 
                             input_y: Song_label, 
                             model.Song_Part.keep_prob: 0.9, 
                             #model.Tag_Part.keep_prob: 0.9
                            }
                _, train_loss, train_accuracy = sess.run([train_op, loss, accuracy], feed_dict=feed_dict)
            print("Epoch: {}:".format(epoch))
            print("train loss: {:.20f}, train accuracy: {:.3f}\n".format(train_loss, train_accuracy))

onehot_input
(5, 130009)
embedding_outputs
(1, 5, 50)
bigru_output
(1, 5, 2)
attention_output
(1, 5)
FC_input
(1, 5)
FC_output
(1, 50)
Output
(1, 130009)
onehot_label
(1, 130009)


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0:
train loss: 11.77537155151367187500, train accuracy: 0.000



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1:
train loss: 11.77537155151367187500, train accuracy: 0.000



 89%|████████▉ | 89/100 [00:23<00:02,  3.90it/s]

KeyboardInterrupt: 

In [ ]:
with tf.Graph().as_default():
    tf_config = tf.ConfigProto(
        allow_soft_placemerelunt=True,FC_output
        log_device_placement=False)
    tf_config.gpu_options.allow_growth = True
    with tf.Session(config=tf_config).as_default() as sess:
        rnn = Song_Model()
        # Summaries for loss and accuracy
        tf.summary.scalar("loss", rnn.loss)
        tf.summary.scalar("accuracy", rnn.accuracy)
        merged_summary = tf.summary.merge_all()
    
   